In [1]:
from typing import List, Union, Set, Tuple, Dict

class Part:
    """
    Either a message or a sublist of rule numbers.
    Examples:
    - message:      part.content == "a"
    - rule numbers: part.content == [1, 2, 3]
    """
    
    def __init__(self, values: List[Union[str, int]]):
        self.is_message = True
        
        self.content = []
        for value in values:
            if isinstance(value, int):
                self.is_message = False
            self.content.append(value)
        
        if self.is_message:
            self.content = "".join(self.content)
                    
    @classmethod
    def from_raw(cls, raw_content: str):

        if raw_content in ['"a"', '"b"']:
            value = raw_content[1:-1]
        else:
            value = [int(entry) for entry in raw_content.split()]
            
        return cls(value)
            
    def __repr__(self):
        return f"Part({self.content})"


class Rule:
    
    def __init__(self, raw_rule: str):
        self.number = int(raw_rule.split(": ")[0])
        self.parts = []
        self.sub_rule_numbers = set()  # Keep track of which sub-rules are in this rule
        
        for sub_rule in raw_rule.split(": ")[1].split(" | "):
            part = Part.from_raw(sub_rule)
            self.parts.append(part)
            if not part.is_message:
                for sub_rule_number in part.content:
                    self.sub_rule_numbers.add(sub_rule_number)

        self._allowed_messages = None  # Set of allowed messages; will be set later.

    def __repr__(self) -> str:
        return f"{self.number:3}: {self.parts}"

    @property
    def is_done(self):
        return len(self.sub_rule_numbers) == 0
    
    @property
    def allowed_messages(self):
        if self._allowed_messages is not None:
            return self._allowed_messages
        
        allowed_messages = set()
        for part in self.parts:
            if part.is_message:
                allowed_messages.add(part.content)
#             else:
#                 raise Exception("Not ready to access allowed messages yet!")
        
        self._allowed_messages = allowed_messages
        return self._allowed_messages

    def is_match(self, message: str) -> bool:        
        return message in self.allowed_messages
    
    def replace(self, other_rule: "Rule") -> bool:
        """Returns whether or not this rule is done after the replacement"""
        # other_rule's parts must be messages.
        for part in other_rule.parts:
            assert part.is_message
        
        if other_rule.number not in self.sub_rule_numbers:
            # Nothing to replace here
            return
        
        self.sub_rule_numbers.remove(other_rule.number)
        updated_parts = []
        
        for part in self.parts:

            if part.is_message:
                # There are no rules to be replaced
                updated_parts.append(part)
                continue
            
            # Part still contains other rule numbers - replace them
            new_contents = [[]]
            for entry in part.content:
                
                if entry != other_rule.number:
                    for new_content in new_contents:
                        new_content.append(entry)
                    continue
                    
                next_new_contents = []
                
                for new_content in new_contents:
                    for other_part in other_rule.parts:
                        next_new_contents.append(new_content + [other_part.content])
                
                new_contents = next_new_contents
            
            updated_parts.extend([Part(values) for values in new_contents])
        self.parts = updated_parts
        
        return self.is_done

def process_rules(raw_rules: str, is_part_two=False) -> Dict[int, Rule]:
    rules = {}
    ready_numbers = set()
    for raw_rule in raw_rules.splitlines():
        rule = Rule(raw_rule)
        if rule.is_done:
            ready_numbers.add(rule.number)
        rules[rule.number] = rule
        
    if is_part_two:
        rules[8] = Rule("8: 42 | 42 8")
        rules[11] = Rule("11: 42 31 | 42 11 31")

    while ready_numbers:
        other_rule = rules[ready_numbers.pop()]

        for rule in rules.values():
            is_done = rule.replace(other_rule)
            if is_done:
                ready_numbers.add(rule.number)

    return rules

# Part 1
This takes a while to run because it's making every possible message that would match Rule 0. I could make it faster (at minimum using Part 2 techniques), but I don't want to right now. :D

In [2]:
filename = "day-19-input.txt"

with open(filename) as file:
    raw_rules, raw_messages = file.read().split("\n\n")
    
messages = raw_messages.splitlines()

rules = process_rules(raw_rules)
rule_zero = rules[0]

count = 0
for message in messages:
    if rule_zero.is_match(message):
        count += 1

print(f"{count} messages completely match rule 0.")

195 messages completely match rule 0.


# Part 2
Takes advantage of the fact that Rule 0 is:
```
0: 8 11
```
and Rules 8 and 11 are:
```
8: 42 | 42 8
11: 42 31 | 42 11 31
```
This is a lot faster than my current implementation of Part 1.

In [3]:
filename = "day-19-input.txt"

with open(filename) as file:
    raw_rules, raw_messages = file.read().split("\n\n")
    
messages = raw_messages.splitlines()

rules = process_rules(raw_rules, is_part_two=True)
rules[0]

  0: [Part([8, 11])]

In [4]:
def get_match_lenth(rule: Rule) -> int:
    lengths = set([len(message) for message in rule.allowed_messages])
    assert len(lengths) == 1
    result = lengths.pop()
    print(result)
    return result

rule_42_match_length = get_match_lenth(rules[42])
rule_31_match_length = get_match_lenth(rules[31])

8
8


In [5]:
def check_against_11(message) -> bool:
    """Returns True if it matches Rule 11. False otherwise."""
    
    # If the message length is too short, we can reject it immediately.
    if len(message) < rule_42_match_length + rule_31_match_length:
        return False
    
    remainder = message
    # Check beginning of message against Rule 42 and end of message against Rule 31
    while (rules[42].is_match(remainder[:rule_42_match_length]) 
           and rules[31].is_match(remainder[-rule_31_match_length:])):
        # Now we only need to check the middle of the message for the potential next iteration
        remainder = remainder[rule_42_match_length:-rule_31_match_length]

        # If we have nothing left to check, then we've found a match!
        if len(remainder) == 0:
            return True
        
        # If the remainder is too short, then we can reject now.
        if len(remainder) < rule_42_match_length + rule_31_match_length:
            return False

count = 0
for message in messages:
    remainder = message
    # Check beginning of message against Rule 8
    while rules[42].is_match(remainder[:rule_42_match_length]):
        # Now we only need to check the rest of the message against Rule 11
        remainder = remainder[rule_42_match_length:]
        if check_against_11(remainder):
            count += 1
            break
        # If it doesn't match Rule 11, we try going through Rule 8 again.

print(f"{count} messages completely match rule 0.")

309 messages completely match rule 0.
